### Imports

In [41]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
# from concurrent.futures import ThreadPoolExecutor
from tqdm.contrib.concurrent import thread_map
pd.set_option('display.max_colwidth', None)

### GET AIRLINES URLS ###

In [42]:
url = 'https://www.pilotjobsnetwork.com/'
max_urls = 5

def get_airlines_urls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href and 'jobs/' in href:
            airline_url = url + href
            airline_name = link.text.strip()
            yield airline_url, airline_name # crée un générateur

### GET INFO FROM PAGES ###

In [47]:
def get_airline_tables(airline_tuple):
    airline_url, airline_name = airline_tuple
    try:
        list_tables = pd.read_html(airline_url)
        salary = list_tables[3].iloc[2:4,1].tolist()
        # iloc iloc[2:4,2] gets you the dates, but remove the salary ... smthing to dig
        return [airline_url, airline_name] + salary
    except Exception as e:
        print(e)
        try:
            return list_tables[3]
        except NameError:
            return airline_url

### MAP IT !! ###

In [44]:
list_salaries = list(thread_map(get_airline_tables, get_airlines_urls(url)))
# print dans un excel le list_salaries

0it [00:00, ?it/s]

'ascii' codec can't encode characters in position 18-19: ordinal not in range(128)
URL can't contain control characters. '/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google' (found at least '\r')


### DF AS OUTPUT ###

In [45]:
df_salary = pd.DataFrame([salary for salary in list_salaries if type(salary) == list])
df_salary

# TO DO :
# DF needs cleaning, columns names, etc
# Check about the NaNs
# See if possible to extract currency (to dble check with meta data, maybe?)
# Add in the DF : "update dates"

,0,1,2,3
0,https://www.pilotjobsnetwork.com/jobs/Corendon_Airlines,5Mar - Turkey (Ma) - Corendon Airlines,6250,5250
1,https://www.pilotjobsnetwork.com/jobs/Hifly,5Mar - Portugal (Ch) - Hifly,"Capt top 240,000â¬","Capt base 220,000â¬"
2,https://www.pilotjobsnetwork.com/jobs/Air_Japan,3Mar - Japan (Le) - Air Japan,$16090,$10930
3,https://www.pilotjobsnetwork.com/jobs/Air_Hamburg_Luftverkehrsgesellschaft_mbH,3Mar - Germany (Fr) - Air Hamburg Luftverkehrsgesellschaft mbH,108.000 Euro / Large Jets 84.000 Euro / Light Jets,84.000 Euro / Large Jets 72.000 Euro / Light Jets
4,https://www.pilotjobsnetwork.com/jobs/Aerowest,3Mar - Germany (Fr) - Aerowest,95.000 EUR,60000
...,...,...,...,...
494,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN
495,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,12Aug - Austria (Fr) - MS AVIATION,NaN,NaN
496,https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation,12Aug - France (Re) - ESMA Aviation,NaN,NaN
497,https://www.pilotjobsnetwork.com/jobs/Compass_Airlines,9Aug - USA (Ma) - Compass Airlines,NaN,NaN


### ERRORS LIST ###

In [46]:
[salary for salary in list_salaries if type(salary) == str]

['https://www.pilotjobsnetwork.com/jobs/Air_CaraÃ¯bes_Atlantique',
 'https://www.pilotjobsnetwork.com/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google']